# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 11:28AM,239118,16,8117637,Sartorius Bioprocess Solutions,Released
1,Jun 8 2022 11:28AM,239119,10,MET211028,"Methapharm, Inc.",Released
2,Jun 8 2022 11:28AM,239119,10,MET211046,"Methapharm, Inc.",Released
3,Jun 8 2022 11:28AM,239119,10,MET211057,"Methapharm, Inc.",Released
4,Jun 8 2022 11:26AM,239112,10,0085745237,ISDIN Corporation,Released
5,Jun 8 2022 11:26AM,239112,10,0085745241,ISDIN Corporation,Released
6,Jun 8 2022 11:26AM,239112,10,0085745242,ISDIN Corporation,Released
7,Jun 8 2022 11:26AM,239112,10,0085745287,ISDIN Corporation,Released
8,Jun 8 2022 11:26AM,239112,10,0085745288,ISDIN Corporation,Released
9,Jun 8 2022 11:26AM,239112,10,0085745296,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
57,239115,Released,1
58,239116,Executing,1
59,239117,Released,2
60,239118,Released,1
61,239119,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239115,NaN,NaN,1.0
239116,NaN,1.0,NaN
239117,NaN,NaN,2.0
239118,NaN,NaN,1.0
239119,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238989,0.0,1.0,0.0
238992,0.0,1.0,0.0
238994,8.0,53.0,2.0
238998,0.0,1.0,15.0
239002,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238989,0,1,0
238992,0,1,0
238994,8,53,2
238998,0,1,15
239002,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238989,0,1,0
1,238992,0,1,0
2,238994,8,53,2
3,238998,0,1,15
4,239002,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238989,,1,
1,238992,,1,
2,238994,8,53,2
3,238998,,1,15
4,239002,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc."
4,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation
62,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC"
64,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc."
65,Jun 8 2022 10:54AM,239115,21,"NBTY Global, Inc."
66,Jun 8 2022 10:35AM,239111,21,"NBTY Global, Inc."
67,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service
68,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions
70,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions,,,1
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc.",,,3
2,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation,,,58
3,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC",,,2
4,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc.",,1,
5,Jun 8 2022 10:54AM,239115,21,"NBTY Global, Inc.",,,1
6,Jun 8 2022 10:35AM,239111,21,"NBTY Global, Inc.",,,1
7,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,,,1
8,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,,,2
9,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions,1,,
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc.",3,,
2,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation,58,,
3,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc.",,1,
5,Jun 8 2022 10:54AM,239115,21,"NBTY Global, Inc.",1,,
6,Jun 8 2022 10:35AM,239111,21,"NBTY Global, Inc.",1,,
7,Jun 8 2022 10:30AM,239110,16,Sartorius Lab Products and Service,1,,
8,Jun 8 2022 10:22AM,239092,16,Sartorius Bioprocess Solutions,2,,
9,Jun 8 2022 10:21AM,239107,18,Mimi Luzon LTD,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions,1,,
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc.",3,,
2,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation,58,,
3,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC",2,,
4,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc.",3.0,NaN,NaN
2,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation,58.0,NaN,NaN
3,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC",2.0,NaN,NaN
4,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 11:28AM,239118,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Jun 8 2022 11:28AM,239119,10,"Methapharm, Inc.",3.0,0.0,0.0
2,Jun 8 2022 11:26AM,239112,10,ISDIN Corporation,58.0,0.0,0.0
3,Jun 8 2022 11:20AM,239117,10,"Nextsource Biotechnology, LLC",2.0,0.0,0.0
4,Jun 8 2022 10:56AM,239116,22,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2390799,147.0,14.0,31.0
102,717254,2.0,1.0,0.0
12,239083,1.0,0.0,0.0
15,1195279,52.0,33.0,0.0
16,717320,4.0,0.0,0.0
18,717214,3.0,0.0,0.0
19,1912364,40.0,10.0,0.0
20,956084,36.0,54.0,8.0
21,2629671,11.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2390799,147.0,14.0,31.0
1,102,717254,2.0,1.0,0.0
2,12,239083,1.0,0.0,0.0
3,15,1195279,52.0,33.0,0.0
4,16,717320,4.0,0.0,0.0
5,18,717214,3.0,0.0,0.0
6,19,1912364,40.0,10.0,0.0
7,20,956084,36.0,54.0,8.0
8,21,2629671,11.0,0.0,0.0
9,22,239116,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,147.0,14.0,31.0
1,102,2.0,1.0,0.0
2,12,1.0,0.0,0.0
3,15,52.0,33.0,0.0
4,16,4.0,0.0,0.0
5,18,3.0,0.0,0.0
6,19,40.0,10.0,0.0
7,20,36.0,54.0,8.0
8,21,11.0,0.0,0.0
9,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,147.0
1,102,Released,2.0
2,12,Released,1.0
3,15,Released,52.0
4,16,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20,21,22
Status,,,,,,,,,,
Completed,31.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
Executing,14.0,1.0,0.0,33.0,0.0,0.0,10.0,54.0,0.0,1.0
Released,147.0,2.0,1.0,52.0,4.0,3.0,40.0,36.0,11.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,31.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,14.0,1.0,0.0,33.0,0.0,0.0,10.0,54.0,0.0,1.0
2,Released,147.0,2.0,1.0,52.0,4.0,3.0,40.0,36.0,11.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20,21,22
0,Completed,31.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0
1,Executing,14.0,1.0,0.0,33.0,0.0,0.0,10.0,54.0,0.0,1.0
2,Released,147.0,2.0,1.0,52.0,4.0,3.0,40.0,36.0,11.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()